#Hillary Talks

I am analysing the HCM dataset to identify trends in the foreign policy along the time. The work is based on the approach by Olalekan but extended to the whole dataset. It is also mainly in python instead of R as Olalekan code, but inserts R code in particular to use the R's countrycode package.

The code below is far from being optimised and it is for "playing" purposes. It was based on python3 and used recent extensions for connecting to R.

In [1]:
import os, sys
import csv
#http://blog.revolutionanalytics.com/2016/01/pipelining-r-python.html
%load_ext rpy2.ipython
import numpy, scipy, pandas
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
from collections import Counter
import datetime
import sqlite3
import zipfile
import nltk, re
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

conn = sqlite3.connect('../input/database.sqlite')
c = conn.cursor()

###Extraction of data (mails and dates) from SQLite to pandas

In [2]:
import pandas.io.sql as psql
sql = "SELECT SUBSTR(MetadataDateSent,1,7) as SentFOIA, ExtractedDateSent as Sent, ExtractedBodyText as EmailBody FROM Emails a WHERE ExtractedBodyText != '' AND MetadataDateSent != '' ORDER BY MetadataDateSent"
df = psql.frame_query(sql, conn)
#df

.../lib/python3.4/site-packages/pandas/io/sql.py:1569: FutureWarning: frame_query is deprecated, use read_sql
  warnings.warn("frame_query is deprecated, use read_sql", FutureWarning)


###Using and preparing R's countrycode dataset

In [3]:
%%R -o countrycode_data_without_atf
library(countrycode)
data(countrycode_data)
#we assume that a country could be referred to as its full name or its ISO3 code
#ec:: OJO with above ^^
#as abbreviation so we get the list of countries
#ec:: list of country names that will be not considered, including USA
#ec:: USA (255) is excluded because we are interested only in foreign policy
countrycode_data_without_atf <- countrycode_data[-c(255,102,83,5,6,9,16,23,31,32,34,36,66,87,88,98,104,105,106,143,117,122,130,154,155,177,183,188,191,201,202,203,204,205,206,207,209,210,212,215,218,224,230,241,248,249,250,266),]



## ec:: the only accepted countries
## ec:: as we will see later, regex is a prepared dataset that includes a pattern to find country.name (NICE!!!)
#countries <- countrycode_data_without_atf[, c("country.name", "regex", "iso2c", "iso3c")]
#countries$other <- NA
## ec:: not clear for me why UK is left apart as "other"
#countries[countries$country.name=="United Kingdom",]$other <- "UK"
##head(countrycode_data_without_atf)
##head(countries)
##countries[countries$country.name=="United Kingdom",]
##countrycode_data_without_atf[with(countrycode_data_without_atf,  grepl("Russia", country.name)),]
##countries[countries$country.name=="Saint Pierre and Miquelon"]


###Parsing the messages in the pandas file to identify possible undesired words

In [4]:
#sw = '|'.join(nltk.corpus.stopwords.words('english') + ["re", "fm", "tv", "la", "al", "ben", "aq"])
#print(sw)
#df_countries.loc[df_countries['country.name']=='Israel',]
def sw(x):
    x = x.split()
    for i,w in enumerate(x):
        #if w in [wrds for wrds in nltk.corpus.words.words() if len(w) <= 3] + ["re", "fm", "tv", "la", "al", "ben", "aq"]:
        if w in nltk.corpus.stopwords.words('english') + ["us", "il", "isr", "re", "fm", "tv", "la", "al", "ben", "aq"]:
            x[i] = 'stopword'
    return ' '.join(x)

df['EmailBody'] = df['EmailBody'].str.lower()
df['EmailBody'] = df['EmailBody'].apply(sw)
df.head()

,SentFOIA,Sent,EmailBody
0,2009-01,"Monday, December 21, 2009 2:33 PM","folks, i've attached stopword latest draft sto..."
1,2009-03,"Thursday, March 19, 2009 12:24 PM",talking points stopword secretary's call stopw...
2,2009-03,,"cheryl mills <cherylmill5 friday, march 20, 20..."
3,2009-03,"Friday, March 20, 20098:05 AM",i've printed stopword stopword stopword give s...
4,2009-03,"Saturday, March 21, 2009 11:21 AM",story stopword stopword stopword good stopword...


###Generation of a list to extract countries found at eac EmailBody
The list is organised by position in the original dataset and consist in a tuple of country name, continent and region per location found at each EmailBody.
In this case, only one record of the country is included if found at least one time in the text.

In [5]:
df_countries = pandas2ri.ri2py(countrycode_data_without_atf)
def match_regex(string, prog):
    return re.match(prog, string)

foundpattern = []
for i, rowdf in df.iterrows():
    p = []
    splitrowdf = rowdf['EmailBody'].split()
    for j,rowc in df_countries.iterrows():
        #first find pattern
        prog = re.compile(rowc['regex'])
        if match_regex(rowdf['EmailBody'], prog):
            p.append((rowc['country.name'], rowc['continent'], rowc['region']))
            continue
        elif rowdf['EmailBody'].find(rowc['country.name'].lower()) != -1:
            p.append((rowc['country.name'], rowc['continent'], rowc['region']))
            continue
        elif rowc['iso2c'].lower() in splitrowdf:
            p.append((rowc['country.name'], rowc['continent'], rowc['region']))
            continue
        elif rowc['iso2c'].lower() == 'gb' and 'uk' in splitrowdf:
            p.append((rowc['country.name'], rowc['continent'], rowc['region']))
            continue
        elif rowc['iso3c'].lower() in splitrowdf:
            p.append((rowc['country.name'], rowc['continent'], rowc['region']))
            continue
    foundpattern.append(p)
foundpattern[:10]

[[],
 [('Korea', 'NA', 'NA'),
  ("Korea, Democratic People's Republic of", 'Asia', 'Eastern Asia')],
 [('Senegal', 'Africa', 'Western Africa')],
 [],
 [('Angola', 'Africa', 'Middle Africa'),
  ('Japan', 'Asia', 'Eastern Asia'),
  ('Madagascar', 'Africa', 'Eastern Africa'),
  ('Morocco', 'Africa', 'Northern Africa')],
 [],
 [],
 [],
 [('Mexico', 'Americas', 'Central America')],
 [('Mexico', 'Americas', 'Central America')]]

###Counting Countries, Continents and Regions: Total

Totals of the number of references to countries, continents and regions for all the messages. If several countries of similar region/continent are found in the same message, those regions and continents are counted only once.

In [6]:
countrycounter = Counter()
continentcounter = Counter()
regioncounter = Counter()
for elem in foundpattern:
    if elem == []: continue
    donecontinentorregion = set()
    for e in elem:
        countrycounter.update([e[0]])
        if e[1] not in donecontinentorregion:
            continentcounter.update([e[1]])
            donecontinentorregion.update([e[1]])
        if e[2] not in donecontinentorregion:
            regioncounter.update([e[2]])
            donecontinentorregion.update([e[2]])

In [7]:
countrycounter

Counter({'Israel': 115, 'Afghanistan': 110, 'China': 97, 'Pakistan': 89, 'Oman': 85, 'Iraq': 79, 'India': 75, 'Peru': 70, 'Libya': 70, 'Angola': 65, 'Korea': 57, 'Germany': 47, 'United Kingdom': 46, 'Egypt': 45, 'Mexico': 40, 'Bangladesh': 39, "Korea, Democratic People's Republic of": 39, 'Brazil': 38, 'Jamaica': 37, 'Turkey': 37, 'Indonesia': 36, 'Japan': 35, 'Ireland': 33, 'France': 29, 'Mali': 28, 'Jordan': 27, 'Honduras': 27, 'Mauritania': 26, 'Morocco': 26, 'Armenia': 23, 'Australia': 23, 'Poland': 22, 'Colombia': 22, 'Cuba': 22, 'Qatar': 22, 'Guyana': 21, 'Panama': 21, 'Canada': 20, 'Ethiopia': 20, 'South Africa': 19, 'Singapore': 19, 'Spain': 18, 'Kenya': 16, 'Yemen': 16, 'Sudan': 15, 'Virgin Islands, U.S.': 15, 'Korea, Republic of': 14, 'Montserrat': 14, 'Sri Lanka': 14, 'Chile': 14, 'Saudi Arabia': 13, 'Estonia': 13, 'Argentina': 13, 'Congo': 12, 'Norway': 12, 'Puerto Rico': 11, 'Nicaragua': 11, 'Italy': 11, 'Yemen Arab Republic': 11, 'Georgia': 10, 'Viet Nam': 10, 'Liechtenst

In [8]:
continentcounter

Counter({'Asia': 616, 'Africa': 356, 'Americas': 353, 'Europe': 254, 'NA': 73, 'Oceania': 52})

In [9]:
regioncounter

Counter({'Western Asia': 331, 'Southern Asia': 241, 'South America': 172, 'Eastern Asia': 152, 'Northern Africa': 151, 'Northern Europe': 126, 'Caribbean': 121, 'Central America': 102, 'Western Africa': 101, 'Middle Africa': 92, 'South-Eastern Asia': 88, 'Western Europe': 81, 'Eastern Africa': 67, 'Southern Europe': 59, 'Eastern Europe': 56, 'Southern Africa': 25, 'Northern America': 25, 'Australia and New Zealand': 24, 'Micronesia': 14, 'Central Asia': 10, 'Melanesia': 7, 'Polynesia': 7})

###Counting Countries, Continents and Regions per Year-Month

A similar counting as above but grouped by year-month.

In [10]:
maxtime = datetime.datetime.strptime(df.SentFOIA.max(),"%Y-%M")
print(maxtime)
mintime = datetime.datetime.strptime(df.SentFOIA.min(),"%Y-%M")
print(mintime)
#http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html
#https://pymotw.com/2/datetime/
#http://stackoverflow.com/questions/4130922/how-to-increment-datetime-by-custom-months-in-python-without-using-library
current = datetime.datetime(mintime.year, 12, 1)
print(current)
next_month = datetime.datetime(mintime.year + int(12 / 12), (int(12 % 12) + 1), 1)
print(next_month)
dates = [mintime]
end = 0
print((maxtime-mintime).days)
while end==0:
    if (maxtime - next_month).days <= 31:
        end = 1
    datetocheck = dates[-1]
    current = datetime.datetime(datetocheck.year, datetocheck.month, 1)
    next_month = datetime.datetime(datetocheck.year + int(datetocheck.month / 12), (int(datetocheck.month % 12) + 1), 1)
    dates.append(next_month)
dates



dateddata = dict([(d,[Counter(),Counter(),Counter()]) for d in dates])


for ii, d in enumerate(df.SentFOIA):
    elem = foundpattern[ii]
    #print(elem)
    current = datetime.datetime(int(d[:4]),int(d[5:7]),1)
    if elem == []: continue
    donecontinentorregion = set()
    for e in elem:
        #print(e)
        dateddata[current][0].update([e[0]])
        if e[1] not in donecontinentorregion:
            dateddata[current][1].update([e[1]])
            donecontinentorregion.update([e[1]])
        if e[2] not in donecontinentorregion:
            dateddata[current][2].update([e[2]])
            donecontinentorregion.update([e[2]])



2014-01-01 00:12:00
2009-01-01 00:01:00
2009-12-01 00:00:00
2010-01-01 00:00:00
1826


A quick report of the top 5 countries in the messaging at each year-month, ordered by number of mentions.

In [11]:
import operator
#sorted(dateddata[list(dateddata)[4]][0].items(), key=operator.itemgetter(1), reverse=True)
for elem in sorted(list(dateddata)):
    #my_list.sort(key=operator.itemgetter(1))
    print(elem,'-----',[x for x in [x[0] for i,x in enumerate(sorted(dateddata[elem][0].items(), key=operator.itemgetter(1), reverse=True)) if i <= 5]])
    #print(elem,'-----',[x for x in list(dateddata[elem][0])])


2009-01-01 00:01:00 ----- []
2009-02-01 00:00:00 ----- []
2009-03-01 00:00:00 ----- ['Mexico', 'Morocco', 'Palestine, State of', 'Oman', 'Senegal', 'China']
2009-04-01 00:00:00 ----- ['Peru', 'Afghanistan', 'Oman', 'China', 'Brazil', 'Korea']
2009-05-01 00:00:00 ----- ['Korea', 'China', 'Angola', "Korea, Democratic People's Republic of", 'Indonesia', 'Sri Lanka']
2009-06-01 00:00:00 ----- ['Honduras', 'China', 'Cuba', 'Angola', 'Korea', 'Korea, Republic of']
2009-07-01 00:00:00 ----- ['Korea', 'India', 'Angola', 'China', 'Peru', 'Korea, Republic of']
2009-08-01 00:00:00 ----- ['Honduras', 'Turkey', 'China', 'Peru', 'Congo', 'India']
2009-09-01 00:00:00 ----- ['Iraq', 'Afghanistan', 'Pakistan', 'China', 'Korea', 'Bangladesh']
2009-10-01 00:00:00 ----- ['Sri Lanka', 'Pakistan', 'Afghanistan', 'Iraq', 'Peru', 'Israel']
2009-11-01 00:00:00 ----- ['Afghanistan', 'Honduras', 'Peru', 'Iraq', 'Pakistan', 'China']
2009-12-01 00:00:00 ----- ['Brazil', 'China', 'Japan', 'Peru', 'India', 'Afghanis

Similar as above per continent.

In [12]:
for elem in sorted(list(dateddata)):
    #my_list.sort(key=operator.itemgetter(1))
    print(elem,'-----',[x for x in [x[0] for i,x in enumerate(sorted(dateddata[elem][1].items(), key=operator.itemgetter(1), reverse=True)) if i <= 6]])
    #print(elem,'-----',[x for x in list(dateddata[elem][0])])


2009-01-01 00:01:00 ----- []
2009-02-01 00:00:00 ----- []
2009-03-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'NA']
2009-04-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'NA']
2009-05-01 00:00:00 ----- ['Asia', 'Africa', 'Americas', 'Europe', 'NA', 'Oceania']
2009-06-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'NA', 'Oceania']
2009-07-01 00:00:00 ----- ['Asia', 'Africa', 'Americas', 'Europe', 'NA', 'Oceania']
2009-08-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'NA', 'Oceania']
2009-09-01 00:00:00 ----- ['Asia', 'Africa', 'Europe', 'Americas', 'NA', 'Oceania']
2009-10-01 00:00:00 ----- ['Asia', 'Africa', 'Europe', 'Americas', 'Oceania', 'NA']
2009-11-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'NA', 'Oceania']
2009-12-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'Oceania', 'NA']
2010-01-01 00:00:00 ----- ['Asia', 'Americas', 'Africa', 'Europe', 'Oceania', 'NA']
2010-02-01 00:00:00 ----- ['Asia', 'Americas', 'Eu

Similar as above per region.

In [13]:
for elem in sorted(list(dateddata)):
    #my_list.sort(key=operator.itemgetter(1))
    print(elem,'-----',[x for x in [x[0] for i,x in enumerate(sorted(dateddata[elem][2].items(), key=operator.itemgetter(1), reverse=True)) if i <= 6]])
    #print(elem,'-----',[x for x in list(dateddata[elem][0])])


2009-01-01 00:01:00 ----- []
2009-02-01 00:00:00 ----- []
2009-03-01 00:00:00 ----- ['Central America', 'Eastern Asia', 'Northern Africa', 'Western Asia', 'Middle Africa', 'Eastern Africa', 'Southern Asia']
2009-04-01 00:00:00 ----- ['South America', 'Eastern Asia', 'Southern Asia', 'Western Asia', 'Northern Africa', 'Central America', 'Western Africa']
2009-05-01 00:00:00 ----- ['Western Asia', 'Southern Asia', 'Eastern Asia', 'Middle Africa', 'South-Eastern Asia', 'South America', 'Caribbean']
2009-06-01 00:00:00 ----- ['Western Asia', 'Eastern Asia', 'Middle Africa', 'Southern Asia', 'Northern Europe', 'South America', 'Central America']
2009-07-01 00:00:00 ----- ['Eastern Asia', 'Southern Asia', 'South America', 'Middle Africa', 'Northern Africa', 'Eastern Africa', 'South-Eastern Asia']
2009-08-01 00:00:00 ----- ['Western Asia', 'South America', 'Southern Asia', 'Eastern Asia', 'Middle Africa', 'Central America', 'Western Africa']
2009-09-01 00:00:00 ----- ['Western Asia', 'Souther

##Conclusions:
* It is easy to see how the attention swifted between months to different countries according to main events that occured during the specific period. For example:
  * 2009 was Aghanistan
  * 2010 was Haiti
  * 2011-2012 was the Arab Spring and Lybia
* The focus of emails concentrated on regions in conflict of different nature. Those regions that were considered in serious conflict that involved USA received more attention. LatinAmerica was first, then regions around Aghanistan and then the region where the Arab Spring occurred.
* The messaging also involved the mentioning of countries that were considered allies to USA, with a strong focus on Israel and Oman, but also Western Europe and OTAN, as well as Pakistan, India, Japan, South Korea and Australia. In Latinamerica, Central America, Peru, Colombia and Brazil seemed to receive special mentioning.
* This results are just a first overview and far to be correct. The reasons of lack of veracity are not only because of the data itself but also the methodology used here. Specifically about the methodology, bear in mind that:
  * there is an unknown number of false positives as well as false negatives when approaching the search using this parsing and the modified country dataset used for the search.
  * some emails contained relevant information about countries in other forms: personalities (which could be directed related to an event in a country or region), or cities/capitals instead of countries (eg. Benghazi).